# Testing phase

In [1]:
import pathlib

#pipeline = pathlib.Path('dataset')
pipeline = pathlib.Path('/kaggle/input/riiid-test-answer-prediction-dataset')

Run the module that contains the source code for the extractors. It's not possible to unpickle them without this.

In [2]:
mod_path = str(pipeline.joinpath('module.py'))
%run $mod_path

Load the parts, which enables the use of certain extractors.

In [3]:
import pandas as pd

parts = pd.read_pickle(pipeline.joinpath('parts.pkl'))

Load the extractors.

In [4]:
import pickle

extractors = []

for path in pipeline.glob('*_extractor.pkl'):
    with open(path, 'rb') as f:
        extractor = pickle.load(f)
        extractors.append(extractor)
        print(extractor)

Load the model.

In [6]:
import lightgbm as lgb

model = lgb.Booster(model_file=str(list(pipeline.glob('*.lgb'))[0]))

Dance on the test set.

In [7]:
import riiideducation

env = riiideducation.make_env()

iter_test = env.iter_test()

prev_group = pd.DataFrame()

for (group, _) in iter_test:
    
    group = group.join(parts, on=['content_type_id', 'content_id'])

    is_question = group['content_type_id'].eq(0)
    questions = group[is_question]
    
    # Add the answer info to the previous group now that's it available
    prev_group['user_answer'] = eval(group.pop('prior_group_responses').iloc[0])
    prev_group['answered_correctly'] = eval(group.pop('prior_group_answers_correct').iloc[0])

    # Update extractors
    for ex in extractors:
        if isinstance(ex, StatefulExtractor):
            ex.update(questions, prev_group)

    # Extract features for the questions
    features = pd.concat((ex.transform(questions) for ex in extractors), axis='columns')
    
    # Align the features according to the order shown during training
    features = features[model.feature_name()]
    
    # Make predictions
    y_pred = model.predict(features)
    
    # Submit
    env.predict(pd.DataFrame(
        {
            'row_id': questions['row_id'],
            'answered_correctly': y_pred
        },
        index=questions.index
    ))
    
    prev_group = group

ModuleNotFoundError: No module named 'riiideducation'